In [ ]:
from matplotlib import pyplot as plt
import numpy as np
%matplotlib inline
import pandas as pd
import seaborn as sns

from matplotlib import cm

import matplotlib as mpl
from scipy import stats
from sklearn.linear_model import LinearRegression

from pathlib import Path
from SyringePumpAux import *


In [ ]:
#Path settings
figureTargetFolder = Path(r"C:\Users\bruno.cruz\Downloads\ExperimentalData\Figures")
dataFolder = Path(r"C:\Users\bruno.cruz\Downloads\ExperimentalData\SingleStepData")

#Global Viz settings
sns.set_context('paper') # darkgrid, white grid, dark, white and ticks
plt.rc('axes', titlesize=18)     # fontsize of the axes title
plt.rc('axes', labelsize=14)    # fontsize of the x and y labels
plt.rc('xtick', labelsize=13)    # fontsize of the tick labels
plt.rc('ytick', labelsize=13)    # fontsize of the tick labels
plt.rc('legend', fontsize=13)    # legend fontsize
plt.rc('font', size=13)          # controls default text sizes
plt.rc("axes.spines", top=False, right=False)

mpl.rcParams['pdf.fonttype'] = 42
mpl.rcParams['ps.fonttype'] = 42
mpl.rcParams['font.family'] = 'Arial'


SeryngePallete = sns.color_palette("deep")

PumpID_colors = {"Pump_2" : SeryngePallete[2],
                "Pump_4" :  SeryngePallete[3]}

color_dict = {"Hamilton_5ml": SeryngePallete[0],
             "Hamilton_10ml": SeryngePallete[1]}
volume_pallete = sns.color_palette("flare", 4)
Volume_colors_dict = {
    "0.33" : volume_pallete[0],
    "2" : volume_pallete[1],
    "5" : volume_pallete[2],
    "10" : volume_pallete[3]
    }

default_fig_size = (6,4)
default_x_scale = (0 , 11)
default_marker_size = 10.


In [ ]:
#HardCoded Variables
SyringeRadiusInfo = {"Hamilton_5ml": 5.15,
                    "Hamilton_10ml": 7.3} #millimeters

StepSizeMultiplier = {"1x":1,
                     "1x2":0.5,
                     "1x4":0.25,
                     "1x8":0.125,
                     "1x16":0.0625} 

MotorFullStepRotation = 1.8 #degrees
ScrewPitch = 0.8 #millimeters per 360 degrees
fullStepStroke =  (MotorFullStepRotation/360.0) * ScrewPitch # millimeters
fullStepVolume_lambdaFNC = lambda r : (np.pi * (r ** 2) ) * fullStepStroke #cubic millimeters (= 1 microliter)


### Load all the data inside the single step folder

In [ ]:
AvailableFolders = [x for x in dataFolder.iterdir() if x.is_dir()]

SyringeData = pd.DataFrame(columns = 
                           ['fullDir', 
                            'fName', 
                            'PumpID', 
                            'SyringeID', 
                            'NSteps', 
                            'StepSize', 
                            'Date'])


for iDir in range(0, len(AvailableFolders)):
    CSVList = [f for f in AvailableFolders[iDir].glob('*.csv')]
    for ii, f in enumerate(CSVList):
        CSVList 
        parsedDict = parseFileName(f) #expects a filename in the format: "Pump_4_Hamilton_5ml_70steps_1x2Step2021-06-23T17_55_46.csv"
        SyringeData = pd.concat([SyringeData, pd.DataFrame({x :[v] for x, v in parsedDict.items()}, columns=SyringeData.columns)])

SyringeData = SyringeData.reset_index(drop=True)
# Add xsection info
SyringeData["SyringeRadius"] = SyringeData["SyringeID"].map(SyringeRadiusInfo)
SyringeData["StepSizeMultiplier"] = SyringeData["StepSize"].map(StepSizeMultiplier)
SyringeData["VolumePerStep"] = SyringeData["StepSizeMultiplier"] * fullStepVolume_lambdaFNC(SyringeData["SyringeRadius"])
SyringeData["VolumePerWaveform"] = SyringeData["VolumePerStep"] * SyringeData["NSteps"]

#Convert the single run dictionary to pandas DF and add it to the original dataFrame.
lambFnc = lambda row : pd.DataFrame(list(
    LoadAndExtractData(row["fullDir"], 
    finalVolumeWindow = np.array([7,10]),
    ratioFinalVolume = 0.95
    ).items())).transpose()

singleDict = SyringeData.apply(lambFnc, axis = 1)

for ii in range(singleDict.shape[0]):
    singleDict[ii].columns = singleDict[ii].iloc[0]
    singleDict[ii] = singleDict[ii].drop(0)
temp_df = singleDict[0]
for ii in range(1,singleDict.shape[0]):
    temp_df = pd.concat([temp_df, singleDict[ii]])
temp_df = temp_df.set_index(SyringeData.index)
SyringeData = SyringeData.join(temp_df, how = 'inner')
print("Check if the data loaded up correctly:")
# Check the output & make sure no TTL errors were deteted
if np.any(SyringeData["TTL_ERROR_FLAG"]):
    print("One of the sessions has something weird in the TTL")
SyringeData.head(2)

In [ ]:
# Averages of averages
SyringeData["MeanDeliveredVolume"] = SyringeData.apply(lambda row : np.nanmean(row["finalVolume"]), axis = 1)
force_0_intercept = True


for syringeID, syringeGroup in SyringeData.groupby("PumpID"):
    grouped = syringeGroup.groupby(['SyringeID','VolumePerWaveform'])
    fig= plt.figure(figsize= default_fig_size)
    ax  = fig.add_subplot(1,1,1)
    
    ## Linear regression
    singleTrialData = np.stack(syringeGroup["finalVolume"].values).flatten()
    volumePerStep = np.stack(syringeGroup.apply(lambda row : row["finalVolume"]*0 + row["VolumePerStep"]*row["NSteps"], axis = 1).values).flatten()
    regModel = LinearRegression(fit_intercept = not(force_0_intercept)).fit(volumePerStep.reshape(-1, 1), singleTrialData.reshape(-1, 1))
    xx = np.array([0,12]).reshape(-1,1)
    yy = regModel.predict(xx)
    if not(force_0_intercept):
        plt.plot(xx, yy, color = PumpID_colors[syringeID], zorder=0, alpha = 0.5, linewidth = 2,
        label = syringeID +r", $R^2=$"+
            str(round(regModel.score(volumePerStep.reshape(-1, 1), singleTrialData.reshape(-1, 1)),3))
            + r", $\beta_1=$" + str(round(regModel.coef_[0,0],2))
            + r", $\beta_0=$" + str(round(regModel.intercept_[0],2))
            )
    else:
        plt.plot(xx, yy, color = PumpID_colors[syringeID], zorder=0, alpha = 0.5,linewidth = 2,
        label = syringeID +r", $R^2=$"+
            str(round(regModel.score(volumePerStep.reshape(-1, 1), singleTrialData.reshape(-1, 1)),3))
            + r", $\beta_1=$" + str(round(regModel.coef_[0,0],2))
            )

    for name, group in grouped:
        alltrials = np.stack(group["finalVolume"].values).flatten()
        plt.errorbar(name[1],
            np.nanmean(alltrials),
            yerr=np.nanstd(alltrials),
            fmt='o', capsize=0, elinewidth = 3, ecolor = PumpID_colors[syringeID],
            mfc = color_dict[name[0]],
            markeredgecolor = PumpID_colors[syringeID],
            label = name[0] + "_" + syringeID, markersize = default_marker_size)

    ax.legend(loc='center left', bbox_to_anchor=(1, 0.5))

    #ax.set_xlim([-1, 10])
    #ax.set_ylim([-0.5, 32.5])
    ax.set_ylabel('Measured displacement (pixels)')
    ax.set_xlabel('Theoretical Volume (uL)')
    ax.set_title("Mean delivered volume " + syringeID)
    ax.set_xlim(default_x_scale)
    fig.savefig(figureTargetFolder / "AveragePerPump_perSyringe_perVolume.pdf")
    #saveFigure(fig, rootSaveFolder = figureTargetFolder)

    plt.show()


In [ ]:
# Averages of averages
SyringeData["MeanDeliveredVolume"] = SyringeData.apply(lambda row : np.nanmean(row["finalVolume"]), axis = 1)

# Add a categorical volume collumn to make things a bit easier later
SyringeData["CategoricalVolume"] = pd.cut(SyringeData["VolumePerWaveform"],bins=[0,0.34,2.1,5.5,11],labels=['0.33','2','5','10'])

uniqueVolumes = pd.unique(SyringeData["CategoricalVolume"])



force_0_intercept = True

for syr, syr_group in SyringeData.groupby('SyringeID'):
    delivered = np.empty( shape=(0) )
    requested = np.empty( shape=(0) )
    for pump, pump_group in syr_group.groupby('PumpID'):
        fig = plt.figure(figsize = default_fig_size)
        ax = fig.add_subplot(1,1,1)
        pumpColor = PumpID_colors[pump]
        for volume, volume_group in pump_group.groupby('VolumePerWaveform'):
            alltrials = np.stack(volume_group["finalVolume"].values).flatten()
            delivered = np.append(delivered, alltrials, axis = 0)
            requested = np.append(requested, alltrials*0 + volume, axis = 0)

            catVolume = volume_group["CategoricalVolume"].iloc[0]

                
            plt.errorbar(volume,
                np.nanmean(alltrials),
                yerr=np.nanstd(alltrials),
                fmt='o', capsize=0, elinewidth = 3, ecolor = Volume_colors_dict[catVolume],
                mfc = color_dict[name[0]],
                markeredgecolor = pumpColor,
                label = name[0] + "_" + pump, markersize = default_marker_size)

        regModel = LinearRegression(fit_intercept = not(force_0_intercept)).fit(requested.reshape(-1, 1), delivered.reshape(-1, 1))
        xx = np.array([0,11]).reshape(-1,1)
        yy = regModel.predict(xx)

        if not(force_0_intercept):
            plt.plot(xx, yy, color = pumpColor, zorder=0, alpha = 0.5, linewidth = 2,
            label = pump +r", $R^2=$"+
                str(round(regModel.score(requested.reshape(-1, 1), delivered.reshape(-1, 1)),3))
                + r", $\beta_1=$" + str(round(regModel.coef_[0,0],2))
                + r", $\beta_0=$" + str(round(regModel.intercept_[0],2))
                )
        else:
            plt.plot(xx, yy, color = pumpColor, zorder=0, alpha = 0.5,linewidth = 2,
            label = pump +r", $R^2=$"+
                str(round(regModel.score(requested.reshape(-1, 1), delivered.reshape(-1, 1)),3))
                + r", $\beta_1=$" + str(round(regModel.coef_[0,0],2))
                )

        plt.xlim(default_x_scale)
        plt.ylim([0, 200])
        plt.ylabel('Measured displacement (pixels)')
        plt.xlabel('Theoretical Volume (uL)')
        plt.title(pump + "_" + syr)
        plt.legend(loc='best')
        saveFigure(fig, rootSaveFolder = figureTargetFolder)
        fig.show()



In [ ]:
SyringeID = "Hamilton_5ml"


for pump, pump_group in syr_group.groupby('PumpID'):
        fig = plt.figure(figsize = default_fig_size)
        ax = fig.add_subplot(1,1,1)
        singlePlotsMask = (pump_group["SyringeID"] == SyringeID)

        for name, group in pump_group[singlePlotsMask].groupby("VolumePerWaveform"):
                catVolume = group["CategoricalVolume"].iloc[0]
                plot_volume_aligned(group, fig = fig, axs = ax,
                        inColor=Volume_colors_dict[catVolume], singleColor=(0,0,0), alpha = 0.25)
                ax.set_title(str(np.round(name,2)) + r"$\mu L $")

        ax.set_ylim(0, 200)
        ax.set_xlim(-1, 5)
        ax.set_title("rawData" + "@" + pump + " with " + SyringeID)
        saveFigure(fig, rootSaveFolder = figureTargetFolder)
        plt.show()


In [ ]:
#Concatenate all trials from multiple runs
fig = plt.figure(figsize = default_fig_size)
ax = fig.add_subplot(1,1,1)

for name, group in SyringeData.groupby(["SyringeID", "VolumePerWaveform", "PumpID"]):
    toPlot = np.stack(group["finalVolume"].values).flatten()
    plt.errorbar(name[1],
    np.nanmean(toPlot),
    yerr=np.nanstd(toPlot),
    fmt='o',
    capsize=0, elinewidth = 3, ecolor = PumpID_colors[name[2]],
    mfc = color_dict[name[0]],
    markersize=10.,
    markeredgecolor = PumpID_colors[name[2]],
    label = name[0] + "\t" + name[2]
    )

#ax.legend()
plt.xlabel('Theoretical Volume (uL)')
plt.ylabel('Measured displacement (pixels)')
plt.title("Average displacement")
plt.legend(loc='center left', bbox_to_anchor=(1, 0.5))
plt.xlim(default_x_scale)
plt.show()

#Plot standard deviation


for SyrID, syrGroup in SyringeData.groupby("SyringeID"):
    fig = plt.figure(figsize = default_fig_size)
    ax = fig.add_subplot(1,1,1) 
    for name, group in syrGroup.groupby(["VolumePerWaveform","PumpID"]):
        toPlot = np.stack(group["finalVolume"].values).flatten()

        catVolume = group["CategoricalVolume"].iloc[0]

        plt.scatter(name[0], 100.0*np.nanstd(toPlot)/np.nanmean(toPlot),
            color = Volume_colors_dict[catVolume],
            edgecolor = PumpID_colors[name[1]],
            label = str(catVolume) + "\t" + name[1], s = default_marker_size**2)

    plt.title("CV_" + SyrID)
    plt.xlabel('Theoretical Volume (uL)')
    plt.ylabel('CV(%)')
    plt.ylim([4,14])
    plt.xlim(default_x_scale)
    saveFigure(fig, rootSaveFolder = figureTargetFolder)

    plt.show()